# Introduction

This notebook demonstrates the use of pre-trained MEGNet models to predict properties.

Author: Tsz Wai Ko (Kenko)
Email: t1ko@ucsd.edu

In [ ]:
import torch
import dgl
import os
from pymatgen.core import Element, Structure, Lattice

## Import megnet related modules
from matgl.graph.converters import get_element_list, Pmg2Graph
from matgl.graph.compute import compute_pair_vector_and_distance
from matgl.models.megnet import MEGNet, MEGNetCalculator


In [ ]:
# Let's set the device first. Either "cuda" or "cpu"
device = torch.device("cpu")

# MP Formation energy 

The pre-trained model is traned on the Materaisl Project mp.2018.6.1.json dataset.

In [ ]:
# load the pre-trained MEGNet model. By default it is the formation energy model.
model = MEGNet.load()
# map the model to CPU or GPU
model = model.to(device)
# This is the structure obtained from the Materials Project.
struct = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.57743067), ["Cu"], [[0, 0, 0]])
# define MEGNet calculator
predictor = MEGNetCalculator(model=model, device=device)
Eform_pred = predictor(struct)
print("The predicted formation energy for a FCC Cu crystal is", float(Eform_pred.detach().numpy()), "eV/atom")


# MP Band gap

The pre-trained model is traned on the Materails Project mp.2019.4.1.json dataset.

In [ ]:
model = MEGNet.load("MP-2019.4.1-BandGap")
# map the model to CPU or GPU
model = model.to(device)
# read structure
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(4.14), ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
# define graph label ("0": PBE, "1": GLLB-SC, "2": HSE, "3": SCAN) for multi-fidelity model
graph_attrs = torch.tensor([0])
# define MEGNet calculator
predictor = MEGNetCalculator(model=model, device=device)
BandGap = predictor(struct, graph_attrs)
print("The predicted PBE BandGap for a CsCl crystal is ", float(BandGap.detach().numpy()), "eV")